# Hybrid VAE

We build a hybrid VAE that uses a quantum circuit to encode the latent space, and a classical neural network to decode it. 

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


# settings
n_qubits = 2
n_layers = 1
dev = qml.device("default.qubit", wires=n_qubits)

## Quantum Encoder

We define a quantum circuit that encodes the input features into a quantum state.

The circuit interfaces with PyTorch, allowing us to use it as part of a larger neural network.

In [ ]:
@qml.qnode(dev, interface="torch")
def quantum_encoder(inputs, weights):
    # Encoding input features as rotations
    for i in range(n_qubits):
        qml.RY(inputs[i], wires=i)
    
    # Variational layers
    for l in range(n_layers):
        for i in range(n_qubits):
            qml.RY(weights[l, i, 0], wires=i)
            qml.RZ(weights[l, i, 1], wires=i)
        qml.CNOT(wires=[0, 1])

    # Measurement
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

This is a PyTorch Module that internally uses the quantum encoder defined above. 

In [ ]:
class QuantumEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        # Variational parameters (torch Parameter)
        self.q_weights = nn.Parameter(0.01 * torch.randn(n_layers, n_qubits, 2))

    def forward(self, x):
        batch_out = []
        for i in range(x.shape[0]):
            res = quantum_encoder(x[i], self.q_weights)
            # Force dtype to float32
            res_torch = torch.tensor(res, dtype=torch.float32)
            batch_out.append(res_torch)
        return torch.stack(batch_out)

## Classical Decoder

This is a classical neural network that decodes the latent space into the output space.

In [ ]:
class ClassicalDecoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 8),
            nn.ReLU(),
            nn.Linear(8, output_dim)
        )

    def forward(self, z):
        return self.model(z)

## Hybrid VAE

Finally, we combine the quantum encoder and classical decoder into a hybrid VAE model.

In [ ]:

class HybridVAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = QuantumEncoder()
        self.decoder = ClassicalDecoder(latent_dim=n_qubits, output_dim=2)

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon

## Training with PyTorch

In [ ]:

# Synthetic data generation
X_data = torch.rand((100, 2), dtype=torch.float32) * np.pi

# Training the Hybrid VAE
model = HybridVAE()
opt = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

for epoch in range(30):
    opt.zero_grad()
    x_recon = model(X_data)
    loss = loss_fn(x_recon, X_data)
    loss.backward()
    opt.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")